# Weaviate에 데이터를 넣기 위한 노트

In [1]:
import os
import json
from dotenv import load_dotenv
import requests
import weaviate
import weaviate.classes as wvc

load_dotenv()

# ENV
API_KEY = os.getenv("GMS_KEY")
EMB_MODEL = os.getenv("EMBEDDING_MODEL")
EMB_URL = os.getenv("EMBEDDING_GMS_URL")

<frozen importlib._bootstrap>:914: ImportWarning: _SixMetaPathImporter.find_spec() not found; falling back to find_module()


## Weaviate와 연결 및 class 생성

In [2]:
# Weaviate랑 연결
client = weaviate.connect_to_custom(
    http_host="localhost",
    http_port=8080,
    grpc_host="localhost",
    grpc_port=50051,
    http_secure=False,
    grpc_secure=False,
    )

In [6]:
existing = client.collections.list_all()
print(existing)
# 데이터 삽입 전 혹시라도 겹칠까봐 삭제.
if ("SingleCounsel" in existing) or ("MultiCounsel" in existing):
    print("이미 존재하고 있음, 삭제 시도")
    client.collections.delete("SingleCounsel")
    client.collections.delete("MultiCounsel")
else:
    print("존재하지 않음")

print(existing)

if "SingleCounsel" not in existing:
    single_collection = client.collections.create(
        name="SingleCounsel",
        description="단일 상담 데이터",
        vector_config=wvc.config.Configure.Vectors.self_provided(),
        properties=[
            wvc.config.Property(name="input", data_type=wvc.config.DataType.TEXT),
            wvc.config.Property(name="output", data_type=wvc.config.DataType.TEXT),
        ],
    )
    print("✅ SingleCounsel 컬렉션 생성 완료")
else:
    print("✅ SingleCounsel 이미 존재. 생성 생략.")

if "MultiCounsel" not in existing:
    client.collections.create(
        name="MultiCounsel",
        description="멀티턴 상담 데이터",
        vector_config=wvc.config.Configure.Vectors.self_provided(),
        properties=[
            wvc.config.Property(name="dialogue", data_type=wvc.config.DataType.TEXT),
            wvc.config.Property(name="summary", data_type=wvc.config.DataType.TEXT),
        ],
    )
    print("✅ MultiCounsel 컬렉션 생성 완료")
else:
    print("✅ MultiCounsel 이미 존재. 생성 생략.")

{}
존재하지 않음
{}
✅ SingleCounsel 컬렉션 생성 완료
✅ MultiCounsel 컬렉션 생성 완료


## 임베딩 함수 (GMS 임베딩), jsonl을 리스트로 열기

In [7]:

def embed(text: str) -> list:
    headers = {
        "Content-type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    }
    payload = {"model": EMB_MODEL, "input": text}

    res = requests.post(EMB_URL, headers=headers, json=payload)
    res.raise_for_status()
    
    result = res.json()["data"][0]["embedding"]

    return [float(v) for v in result]  # 1536 or 3072 dim

def load_jsonl(path):
    with open(path, "r", encoding="utf-8") as f:
        return [json.loads(line) for line in f if line.strip()]
    
import weaviate
import weaviate.classes as wvc
import numpy as np

# 벡터 검증 및 변환 함수
def validate_and_convert_vector(vector):
    """벡터를 list[float]로 변환"""
    if vector is None:
        return None
    
    # numpy 배열 → list 변환
    if isinstance(vector, np.ndarray):
        vector = vector.tolist()
    
    # list로 변환 확인
    if not isinstance(vector, list):
        raise TypeError(f"Vector must be list, got {type(vector)}")
    
    # 차원 확인
    if len(vector) == 0:
        raise ValueError("Vector cannot be empty")
    
    # float 검증
    vector = [float(v) for v in vector]
    
    return vector

### 단일 상담 데이터 삽입(50개)

In [8]:
# 수정된 코드
single_data = load_jsonl("./total_kor_counsel_bot.jsonl")[:50]

col = client.collections.get("SingleCounsel")

for idx, d in enumerate(single_data):
    try:
        # 1. 임베딩 생성 (에러 처리 포함)
        vector_embedding = validate_and_convert_vector(embed(d["input"]))
        
        # 2. 벡터 검증
        if not vector_embedding or len(vector_embedding) == 0:
            print(f"❌ Row {idx}: 벡터가 비어있습니다")
            continue
            
        # 3. 데이터 객체 생성
        data_object = {
            "input": d["input"].strip(),
            "output": d["output"].strip()
        }
        
        # 4. Insert 및 반환값 확인
        uuid = single_collection.data.insert(
            properties=data_object,
            vector=vector_embedding
        )
        print(f"✅ Row {idx}: 저장 완료 (UUID: {uuid})")
        
    except Exception as e:
        print(f"❌ Row {idx} 에러: {str(e)}")
        continue

print("✅ SingleCounsel 업로드 완료")



✅ Row 0: 저장 완료 (UUID: 3d6eb4c9-6e2b-4281-ab15-fd4a9d5ffaee)
✅ Row 1: 저장 완료 (UUID: 115a219e-6489-4826-a945-d2a27e7ae1fe)
✅ Row 2: 저장 완료 (UUID: 7de9bbde-35d9-486e-9ddc-dac214bdf07e)
✅ Row 3: 저장 완료 (UUID: 6974c1b6-d325-4a89-b0be-9cd52e387d34)
✅ Row 4: 저장 완료 (UUID: ac955319-5626-4dfe-9678-95fe8ebb3f1a)
✅ Row 5: 저장 완료 (UUID: 48326b8e-9285-44fb-8aff-0876b2cf3eba)
✅ Row 6: 저장 완료 (UUID: b0f7307b-cea0-425a-bd60-49c03bbbf045)
✅ Row 7: 저장 완료 (UUID: 80ed77d4-d2a2-448c-a0ac-a89899e9e2ff)
✅ Row 8: 저장 완료 (UUID: f429a2a6-8565-4839-9d23-8cfda09f63d9)
✅ Row 9: 저장 완료 (UUID: 6a1ae025-4a81-48c9-8144-e7434f06758c)
✅ Row 10: 저장 완료 (UUID: eb235588-245b-4d37-983d-d809b196e840)
✅ Row 11: 저장 완료 (UUID: bf91c946-b033-4655-a586-fca282e1e4ef)
✅ Row 12: 저장 완료 (UUID: 45e85f0e-b1ea-4ea2-98d5-c43872248825)
✅ Row 13: 저장 완료 (UUID: 09e5da63-b3b9-4a9b-8409-27a8b8b707c1)
✅ Row 14: 저장 완료 (UUID: bf6d3487-2c8f-4a21-ab63-a3a6b1385bf3)
✅ Row 15: 저장 완료 (UUID: eeb25c67-4c9a-423f-be38-15e0ba8cf315)
✅ Row 16: 저장 완료 (UUID: e11d25bf-07

In [9]:
# 4. 저장 확인
result = single_collection.query.fetch_objects(limit=1)
if result.objects and result.objects[0].vector:
    print(f"🎉 벡터 저장 성공! Vector length: {len(result.objects[0].vector)}")
else:
    print("❌ 벡터 여전히 저장 안 됨")


❌ 벡터 여전히 저장 안 됨


### 멀티턴 상담 데이터 삽입

In [10]:
# # 이게 기존의 데이터 삽입 형식

# multi_collection = client.collections.get("MultiCounsel")
# multi_data = load_jsonl("./total_kor_multiturn_counsel_bot.jsonl")[:50]

# multi_objs = []
# for turns in multi_data:
#     dialogue = "\n".join(f"{t['speaker']}: {t['utterance']}" for t in turns)
#     print("현재 업로드 하는 내용은 : ", dialogue)
#     vec = embed(dialogue)

#     multi_objs.append(
#         wvc.data.DataObject(
#             properties={"dialogue": dialogue, "summary": ""},
#             vector=vec,
#         )
#     )

# multi_collection.data.insert_many(multi_objs)
# print("✅ MultiCounsel 업로드 완료")

In [15]:
multi_collection = client.collections.get("MultiCounsel")
multi_data = load_jsonl("./total_kor_multiturn_counsel_bot.jsonl")[:50]

for idx, d in enumerate(multi_data):
    try:
        # 1. 임베딩 생성 (에러 처리 포함)
        vector_embedding = validate_and_convert_vector(embed(d["dialogue"]))
        
        # 2. 벡터 검증
        if not vector_embedding or len(vector_embedding) == 0:
            print(f"❌ Row {idx}: 벡터가 비어있습니다")
            continue
            
        # 3. 데이터 객체 생성
        data_object = {
            "dialogue": d["dialogue"].strip(),
        }
        
        # 계속 안돼서 DataObject를 사용하여 삽입 시도.
        obj = multi_collection.data.insert(
            properties=data_object,
            vector=vector_embedding
            )
        
        # 4. Insert 및 반환값 확인
        print(f"✅ Row {idx}: 저장 완료 (UUID: {uuid})")
        
    except Exception as e:
        print(f"❌ Row {idx} 에러: {str(e)}")
        continue

print("✅ SingleCounsel 업로드 완료")

# 4. 저장 확인
result = multi_collection.query.fetch_objects(limit=1)
if result.objects and result.objects[0].vector:
    print(f"🎉 벡터 저장 성공! Vector length: {len(result.objects[0].vector)}")
else:
    print("❌ 벡터 여전히 저장 안 됨")

❌ Row 0 에러: 400 Client Error: Bad Request for url: https://gms.ssafy.io/gmsapi/api.openai.com/v1/embeddings
❌ Row 1 에러: 400 Client Error: Bad Request for url: https://gms.ssafy.io/gmsapi/api.openai.com/v1/embeddings
❌ Row 2 에러: 400 Client Error: Bad Request for url: https://gms.ssafy.io/gmsapi/api.openai.com/v1/embeddings
❌ Row 3 에러: 400 Client Error: Bad Request for url: https://gms.ssafy.io/gmsapi/api.openai.com/v1/embeddings
❌ Row 4 에러: 400 Client Error: Bad Request for url: https://gms.ssafy.io/gmsapi/api.openai.com/v1/embeddings
❌ Row 5 에러: 400 Client Error: Bad Request for url: https://gms.ssafy.io/gmsapi/api.openai.com/v1/embeddings
❌ Row 6 에러: 400 Client Error: Bad Request for url: https://gms.ssafy.io/gmsapi/api.openai.com/v1/embeddings
❌ Row 7 에러: 400 Client Error: Bad Request for url: https://gms.ssafy.io/gmsapi/api.openai.com/v1/embeddings
❌ Row 8 에러: 400 Client Error: Bad Request for url: https://gms.ssafy.io/gmsapi/api.openai.com/v1/embeddings
❌ Row 9 에러: 400 Client Error

KeyboardInterrupt: 

### 확인 및 Weaviate client 종료

In [18]:
import weaviate
print(weaviate.__version__)

4.17.0


In [1]:
from weaviate.classes.query import MetadataQuery

call = client.collections.get("MultiCounsel")
res = call.query.fetch_objects(limit = 10, include_vector=True)
print(len(res.objects))

for i, obj in enumerate(res.objects, start=1):
    print(f"\n=== Object {i} ===")
    print("ID:", obj.uuid)
    print("properties:", obj.properties)
    print("vector exists:", obj.vector is not None)
    print("vector length:", obj.vector)

<frozen importlib._bootstrap>:914: ImportWarning: _SixMetaPathImporter.find_spec() not found; falling back to find_module()


NameError: name 'client' is not defined

In [ ]:
test_result = single_collection.query.fetch_objects(limit=1)

if test_result.objects and test_result.objects[0].vector:
    print("🎉 Vector length:", len(test_result.objects[0].vector))
else:
    print("❌ Vector가 저장되지 않았습니다.")


client.close()
print("🔌 Weaviate 연결 종료")